In [1]:
import sys
sys.path.append('../')
import game
import layout as lay
import capture
import environment
import captureGraphicsDisplay
import textDisplay
import stable_baselines3 as sb3

rules = capture.CaptureRules()

In [2]:
# Runs an entire game, you can use this for reference/understanding/debugging purposes.
# Other than that, it has no use.
def run_game():
    layout = lay.get_layout('../layouts/defaultCapture.lay')

    rl_agents = capture.load_agents(True, '../baselineTeam', [])
    enemy_agents = capture.load_agents(True, '../baselineTeam', [])

    red_agents = enemy_agents
    blue_agents = rl_agents
    agents = sum([list(el) for el in zip(red_agents, blue_agents)], [])
    game = rules.new_game(layout = layout, agents = agents, display = textDisplay.NullGraphics(), length = 1200, mute_agents = False, catch_exceptions = False)
    agent_index = game.starting_index
    num_agents = 4

    states_explored = 0
    while not game.game_over:
        agent = agents[agent_index]
        
        observation = agent.observation_function(game.state.deep_copy())
        action = agent.get_action(observation)
        # print(game.state)
        states_explored += 1
        # print(states_explored)
        game.state = game.state.generate_successor(agent_index, action)
        game.move_history.append((agent_index, action))
        
        rules.process(game.state, game)
        agent_index = (agent_index + 1) % num_agents

In [3]:
env = environment.PacmanEnv(rules, max_episode_length = 1200)

rl = sb3.DQN('MultiInputPolicy', env, tensorboard_log = './logs/', verbose = 1, learning_starts = 1000, buffer_size = 10_000, target_update_interval = 1000, exploration_final_eps = 0.1)

rl.learn(total_timesteps= 100_000)

Loading agent team: /home/adrian/facultate/emai/AS/pacman-agent/pacman-contest/src/contest/baselineTeam.py
	Arguments: {}
Loading agent team: /home/adrian/facultate/emai/AS/pacman-agent/pacman-contest/src/contest/baselineTeam.py
	Arguments: {}
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Blue team starts
We are team BLUE


TypeError: float() argument must be a string or a real number, not 'dict'

In [ ]:
layout = lay.get_layout('../layouts/defaultCapture.lay')

rl_agents = capture.load_agents(False, 'myTeamRL', [])
enemy_agents = capture.load_agents(True, '../baselineTeam', [])

rl_agents[0].rl = rl
rl_agents[1].rl = rl

red_agents = enemy_agents
blue_agents = rl_agents
agents = sum([list(el) for el in zip(red_agents, blue_agents)], [])

display = captureGraphicsDisplay.PacmanGraphics(
    redTeam = '../baselineTeam',
    redName = 'Red',
    blueTeam = 'myTeamTest',
    blueName = 'Blue',
    frameTime = 0
    )
NUM_ACTIONS = 4
game = rules.new_game(layout = layout, agents = agents, display = display, length = 1200, mute_agents = False, catch_exceptions = False)
game.run()

Loading agent team: /home/adrian/facultate/emai/AS/pacman-agent/pacman-contest/src/contest/rl-pacman/myTeamRL.py
	Arguments: {}
Loading agent team: /home/adrian/facultate/emai/AS/pacman-agent/pacman-contest/src/contest/baselineTeam.py
	Arguments: {}
Red team starts
Time is up.
Tie game!
